# Business Problem
Your company now sees all the big companies creating original video content and they want to get in on the fun. They have decided to create a new movie studio, but they don’t know anything about creating movies. You are charged with exploring what types of films are currently doing the best at the box office. You must then translate those findings into actionable insights that the head of your company's new movie studio can use to help decide what type of films to create.

# Objectives
Analyze historical movie data to:

-Understand what genres and attributes are associated with high box office performance.

-Identify trends in movie production and profitability.

-Make data-driven recommendations on the types of movies to produce.


## **Step 1**: Data Collection

-Import the necessary libraries

-Load movie data from a SQL database or CSV files.

-Explore the available tables and columns.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sqlite3
from scipy import stats

### (i) Load the im.db database and inspect it

In [2]:
# Load the im.db database and read the tables using pandas
conn = sqlite3.connect(r"C:\Users\user\Desktop\phase_2_project\Phase-2-Project\zippedData\im.db\im.db")
cur = conn.cursor()
pd.read_sql('''SELECT name FROM sqlite_master WHERE type='table';''', conn)

,name
0,movie_basics
1,directors
2,known_for
3,movie_akas
4,movie_ratings
5,persons
6,principals
7,writers


In [3]:
# Read from the movie_basics table
movie_basics_df = pd.read_sql(''' 
            SELECT *
            FROM movie_basics;
            ''',conn)
movie_basics_df.head()

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"


In [4]:
# Read from the movie_ratings table
movie_ratings_df = pd.read_sql('''
                               SELECT *
                               FROM movie_ratings;
                               ''',conn)
movie_ratings_df.head()

,movie_id,averagerating,numvotes
0,tt10356526,8.3,31
1,tt10384606,8.9,559
2,tt1042974,6.4,20
3,tt1043726,4.2,50352
4,tt1060240,6.5,21


In [5]:
# Read from the movie_akas table
movie_akas_df = pd.read_sql(''' SELECT *
                            FROM movie_akas;
                            ''', conn)
movie_akas_df.head()

,movie_id,ordering,title,region,language,types,attributes,is_original_title
0,tt0369610,10,Джурасик свят,BG,bg,None,None,0.0
1,tt0369610,11,Jurashikku warudo,JP,None,imdbDisplay,None,0.0
2,tt0369610,12,Jurassic World: O Mundo dos Dinossauros,BR,None,imdbDisplay,None,0.0
3,tt0369610,13,O Mundo dos Dinossauros,BR,None,None,short title,0.0
4,tt0369610,14,Jurassic World,FR,None,imdbDisplay,None,0.0


In [6]:
directors_df = pd.read_sql(''' 
                           SELECT *
                           FROM directors;
                           ''',conn)
directors_df

,movie_id,person_id
0,tt0285252,nm0899854
1,tt0462036,nm1940585
2,tt0835418,nm0151540
3,tt0835418,nm0151540
4,tt0878654,nm0089502
...,...,...
291169,tt8999974,nm10122357
291170,tt9001390,nm6711477
291171,tt9001494,nm10123242
291172,tt9001494,nm10123248


In [7]:
persons_df = pd.read_sql(''' 
                        SELECT *
                        FROM persons;
                        ''', conn)
persons_df

,person_id,primary_name,birth_year,death_year,primary_profession
0,nm0061671,Mary Ellen Bauder,NaN,NaN,"miscellaneous,production_manager,producer"
1,nm0061865,Joseph Bauer,NaN,NaN,"composer,music_department,sound_department"
2,nm0062070,Bruce Baum,NaN,NaN,"miscellaneous,actor,writer"
3,nm0062195,Axel Baumann,NaN,NaN,"camera_department,cinematographer,art_department"
4,nm0062798,Pete Baxter,NaN,NaN,"production_designer,art_department,set_decorator"
...,...,...,...,...,...
606643,nm9990381,Susan Grobes,NaN,NaN,actress
606644,nm9990690,Joo Yeon So,NaN,NaN,actress
606645,nm9991320,Madeline Smith,NaN,NaN,actress
606646,nm9991786,Michelle Modigliani,NaN,NaN,producer


In [8]:
principals_df = pd.read_sql(''' 
                        SELECT *
                        FROM principals;
                        ''', conn)
principals_df

,movie_id,ordering,person_id,category,job,characters
0,tt0111414,1,nm0246005,actor,None,"[""The Man""]"
1,tt0111414,2,nm0398271,director,None,None
2,tt0111414,3,nm3739909,producer,producer,None
3,tt0323808,10,nm0059247,editor,None,None
4,tt0323808,1,nm3579312,actress,None,"[""Beth Boothby""]"
...,...,...,...,...,...,...
1028181,tt9692684,1,nm0186469,actor,None,"[""Ebenezer Scrooge""]"
1028182,tt9692684,2,nm4929530,self,None,"[""Herself"",""Regan""]"
1028183,tt9692684,3,nm10441594,director,None,None
1028184,tt9692684,4,nm6009913,writer,writer,None


In [9]:
# Join the tables picking only the relevant columns

sql = """
SELECT
  mb.movie_id,
  mb.start_year,
  mb.genres,
  mr.averagerating,
  ma.region,
  ma.language,
  p.primary_name AS director_name,
  pr.category, 
  ma.region AS movie_region,
  ma.language AS movie_language
FROM movie_basics AS mb

-- Join with movie_ratings table
INNER JOIN movie_ratings AS mr
  ON mb.movie_id = mr.movie_id

-- Join with movie_akas table
INNER JOIN (
  SELECT movie_id, region, language
  FROM movie_akas
) AS ma
  ON mb.movie_id = ma.movie_id

-- Join with directors table to get person_id for directors
INNER JOIN directors AS d
  ON mb.movie_id = d.movie_id

-- Join with persons table to get the director name
INNER JOIN persons AS p
  ON d.person_id = p.person_id

-- Join with principals table to get category
INNER JOIN principals AS pr
  ON mb.movie_id = pr.movie_id
;
"""

# 3. Execute the query and load into a pandas DataFrame
merged_sql_df = pd.read_sql_query(sql, conn)

# Inspect the first few rows of the merged data
merged_sql_df.head()


,movie_id,start_year,genres,averagerating,region,language,director_name,category,movie_region,movie_language
0,tt0063540,2013,"Action,Crime,Drama",7.0,None,None,Harnam Singh Rawail,actor,None,None
1,tt0063540,2013,"Action,Crime,Drama",7.0,None,None,Harnam Singh Rawail,actor,None,None
2,tt0063540,2013,"Action,Crime,Drama",7.0,None,None,Harnam Singh Rawail,actor,None,None
3,tt0063540,2013,"Action,Crime,Drama",7.0,None,None,Harnam Singh Rawail,actress,None,None
4,tt0063540,2013,"Action,Crime,Drama",7.0,None,None,Harnam Singh Rawail,composer,None,None


In [10]:
# check the information about the dataframe
merged_sql_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6939751 entries, 0 to 6939750
Data columns (total 10 columns):
 #   Column          Dtype  
---  ------          -----  
 0   movie_id        object 
 1   start_year      int64  
 2   genres          object 
 3   averagerating   float64
 4   region          object 
 5   language        object 
 6   director_name   object 
 7   category        object 
 8   movie_region    object 
 9   movie_language  object 
dtypes: float64(1), int64(1), object(8)
memory usage: 529.5+ MB


In [11]:
# Inspect the shape of the dataframe
rows,columns = merged_sql_df.shape
print(f' The dataframe has {rows} rows and {columns} columns')

 The dataframe has 6939751 rows and 10 columns


In [12]:
# check for duplicates
duplicates = merged_sql_df.duplicated()
duplicate_count = duplicates.sum()
if duplicate_count > 0:
    print(f"Number of duplicated rows: {duplicate_count}")
else:
    print("No duplicated rows found.")

Number of duplicated rows: 5450981


In [13]:
# Drop duplicates and recheck the shape
merged_sql_df.drop_duplicates(inplace=True)
rows, columns = merged_sql_df.shape
print(f'The mon-duplicate dataframe has {rows} rows and {columns} columns')

The mon-duplicate dataframe has 1488770 rows and 10 columns


In [14]:
# check for missing values
merged_sql_df.isna().sum()

movie_id                0
start_year              0
genres               5714
averagerating           0
region             227991
language          1271880
director_name           0
category                0
movie_region       227991
movie_language    1271880
dtype: int64

In [15]:
# check for percentage of each missing column so as to determine the methods of cleaning
missing_percent_per_column = merged_sql_df.isnull().mean() * 100
missing_percent_per_column

movie_id           0.000000
start_year         0.000000
genres             0.383807
averagerating      0.000000
region            15.314051
language          85.431598
director_name      0.000000
category           0.000000
movie_region      15.314051
movie_language    85.431598
dtype: float64

In [16]:
# Drop duplicated columns

# sanity‐check that they really are identical
assert merged_sql_df['language'].equals(merged_sql_df['movie_language']), "language vs movie_language differ!"
assert merged_sql_df['region'].equals(merged_sql_df['movie_region']),     "region vs movie_region differ!"

#drop the duplicated columns
df = merged_sql_df.drop(columns=['movie_language', 'movie_region'],inplace=True)


In [17]:
missing_percent_per_column1 = merged_sql_df.isnull().mean() * 100
missing_percent_per_column1

movie_id          0.000000
start_year        0.000000
genres            0.383807
averagerating     0.000000
region           15.314051
language         85.431598
director_name     0.000000
category          0.000000
dtype: float64

Two columns (genres, region, language) require cleaning;

- we replace the null values of genre and region with the mode

- we we drop the language column since it has very many missing values


In [18]:
# create a copy of the original dataframe
merged_sql_df_copy = merged_sql_df.copy()

In [19]:
# Replace missing values in the region column with mode
column_to_fill = ['genres', 'region']
merged_sql_df_copy[column_to_fill] = merged_sql_df_copy[column_to_fill].fillna(merged_sql_df_copy[column_to_fill].mode().iloc[0])


In [20]:
# Drop the language column
merged_sql_df_copy.drop(columns=['language'],inplace=True)

In [21]:
# Recheck if the dataframe is fully clean
merged_sql_df_copy.isna().sum()

movie_id         0
start_year       0
genres           0
averagerating    0
region           0
director_name    0
category         0
dtype: int64

### (ii) Load the rotten tomatoes dataset using pandas,inspect it and clean it

In [23]:


rotten_tomatoes_df = pd.read_csv(
    r"C:\Users\user\Desktop\phase_2_project\Phase-2-Project\zippedData\rt.movie_info.tsv\rt.movie_info.tsv",
    sep='\t',
    encoding='utf-8'
)

rotten_tomatoes_df.head()



,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio
0,1,"This gritty, fast-paced, and innovative police...",R,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman,"Oct 9, 1971","Sep 25, 2001",NaN,NaN,104 minutes,NaN
1,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012","Jan 1, 2013",$,"600,000",108 minutes,Entertainment One
2,5,Illeana Douglas delivers a superb performance ...,R,Drama|Musical and Performing Arts,Allison Anders,Allison Anders,"Sep 13, 1996","Apr 18, 2000",NaN,NaN,116 minutes,NaN
3,6,Michael Douglas runs afoul of a treacherous su...,R,Drama|Mystery and Suspense,Barry Levinson,Paul Attanasio|Michael Crichton,"Dec 9, 1994","Aug 27, 1997",NaN,NaN,128 minutes,NaN
4,7,NaN,NR,Drama|Romance,Rodney Bennett,Giles Cooper,NaN,NaN,NaN,NaN,200 minutes,NaN


In [24]:
# check for information about the dataframe
rotten_tomatoes_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1560 entries, 0 to 1559
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            1560 non-null   int64 
 1   synopsis      1498 non-null   object
 2   rating        1557 non-null   object
 3   genre         1552 non-null   object
 4   director      1361 non-null   object
 5   writer        1111 non-null   object
 6   theater_date  1201 non-null   object
 7   dvd_date      1201 non-null   object
 8   currency      340 non-null    object
 9   box_office    340 non-null    object
 10  runtime       1530 non-null   object
 11  studio        494 non-null    object
dtypes: int64(1), object(11)
memory usage: 146.4+ KB


In [25]:
# Inspect the shape of the dataframe
rows, columns = rotten_tomatoes_df.shape
print(f'Number of columns:{columns}')
print(f'Number or rows: {rows}')

Number of columns:12
Number or rows: 1560


In [26]:
# check for duplicates
duplicates = rotten_tomatoes_df.duplicated()
duplicate_count = duplicates.sum()
if duplicate_count == 0:
    print('No duplicated values')
else:
    print(f'Duplicated values count: {duplicate_count}')

No duplicated values


In [27]:
# check for mising values
rotten_tomatoes_df.isna().sum()

id                 0
synopsis          62
rating             3
genre              8
director         199
writer           449
theater_date     359
dvd_date         359
currency        1220
box_office      1220
runtime           30
studio          1066
dtype: int64

In [28]:
# check for percentage of each missing column so as to determine the methods of cleaning
missing_percent_per_column2 = rotten_tomatoes_df.isnull().mean() * 100
missing_percent_per_column2



id               0.000000
synopsis         3.974359
rating           0.192308
genre            0.512821
director        12.756410
writer          28.782051
theater_date    23.012821
dvd_date        23.012821
currency        78.205128
box_office      78.205128
runtime          1.923077
studio          68.333333
dtype: float64

In [29]:
# create a copy of the original dataframe
rotten_tomatoes_df_copy = rotten_tomatoes_df.copy()

In [30]:
# For columns having missing values more than 50% we drop them 
rotten_dropped = rotten_tomatoes_df_copy.drop(columns=['currency','box_office','studio'],inplace=True)

In [31]:
# For columns having missing values between 10%-49% we drop the rows with missing values
rotten_tomatoes_df_copy.dropna(subset = ['director', 'writer', 'theater_date','dvd_date'], inplace=True)

In [32]:
# For columns with less than 10% of missing values, replace the missing values with mode
columns_to_fill = ['synopsis', 'rating', 'genre', 'runtime']
rotten_tomatoes_df_copy[columns_to_fill] = rotten_tomatoes_df_copy[columns_to_fill].fillna(rotten_tomatoes_df_copy[columns_to_fill].mode().iloc[0])

In [33]:
# check if the dataframe is fully clean
rotten_tomatoes_df_copy.isna().sum()


id              0
synopsis        0
rating          0
genre           0
director        0
writer          0
theater_date    0
dvd_date        0
runtime         0
dtype: int64

The dataframe is now clean and ready for analysis

### (iii) Load the Numbers Dataframe, inspect it and clean it

In [34]:
numbers_df = pd.read_csv(
    r"C:\Users\user\Desktop\phase_2_project\Phase-2-Project\zippedData\rt.reviews.tsv\rt.reviews.tsv",
    sep='\t',  
    encoding='latin1' 
)

numbers_df

,id,review,rating,fresh,critic,top_critic,publisher,date
0,3,A distinctly gallows take on contemporary fina...,3/5,fresh,PJ Nabarro,0,Patrick Nabarro,"November 10, 2018"
1,3,It's an allegory in search of a meaning that n...,NaN,rotten,Annalee Newitz,0,io9.com,"May 23, 2018"
2,3,... life lived in a bubble in financial dealin...,NaN,fresh,Sean Axmaker,0,Stream on Demand,"January 4, 2018"
3,3,Continuing along a line introduced in last yea...,NaN,fresh,Daniel Kasman,0,MUBI,"November 16, 2017"
4,3,... a perverse twist on neorealism...,NaN,fresh,NaN,0,Cinema Scope,"October 12, 2017"
...,...,...,...,...,...,...,...,...
54427,2000,The real charm of this trifle is the deadpan c...,NaN,fresh,Laura Sinagra,1,Village Voice,"September 24, 2002"
54428,2000,NaN,1/5,rotten,Michael Szymanski,0,Zap2it.com,"September 21, 2005"
54429,2000,NaN,2/5,rotten,Emanuel Levy,0,EmanuelLevy.Com,"July 17, 2005"
54430,2000,NaN,2.5/5,rotten,Christopher Null,0,Filmcritic.com,"September 7, 2003"
